In [2]:
from PIL import Image
import numpy as np

def crop_flag(image_path):
    # Load the image using Pillow
    img = Image.open(image_path)

    if img is None:
        print("Error: Image not found!")
        return

    # Convert the image to RGB (Pillow uses RGBA by default)
    img_rgb = img.convert('RGB')

    # Convert the image to a Numpy array for easier manipulation
    img_array = np.array(img_rgb)

    # Define thresholds for red and white colors in RGB
    # Red: R > 100, G < 50, B < 50 (just an approximation)
    red_mask = (img_array[:, :, 0] > 100) & (img_array[:, :, 1] < 50) & (img_array[:, :, 2] < 50)

    # White: R > 200, G > 200, B > 200 (approximation for white)
    white_mask = (img_array[:, :, 0] > 200) & (img_array[:, :, 1] > 200) & (img_array[:, :, 2] > 200)

    # Combine red and white masks
    combined_mask = red_mask | white_mask

    # Find the bounding box for the region of interest (the flag area)
    rows = np.any(combined_mask, axis=1)
    cols = np.any(combined_mask, axis=0)

    # Get the bounding box coordinates
    top, bottom = np.where(rows)[0][[0, -1]]
    left, right = np.where(cols)[0][[0, -1]]

    # Crop the image based on the bounding box
    cropped_img = img_array[top:bottom+1, left:right+1]

    # Get the width of the cropped image
    cropped_width = cropped_img.shape[1]

    # Calculate the cut portion (1/4th from both left and right)
    cut_width = cropped_width // 4
    cropped_img_cut = cropped_img[:, cut_width:-cut_width]  # Remove 1/4th from left and right

    # Convert the cropped image back to a Pillow Image for display and saving
    cropped_pil_img_cut = Image.fromarray(cropped_img_cut)

    # Apply the cropping process again on the cropped image
    # Convert the cropped image to a Numpy array
    cropped_img_array = np.array(cropped_pil_img_cut)

    # Red and White color masks for the new cropped image
    red_mask = (cropped_img_array[:, :, 0] > 100) & (cropped_img_array[:, :, 1] < 50) & (cropped_img_array[:, :, 2] < 50)
    white_mask = (cropped_img_array[:, :, 0] > 200) & (cropped_img_array[:, :, 1] > 200) & (cropped_img_array[:, :, 2] > 200)

    # Combine red and white masks for the cropped image
    combined_mask_cropped = red_mask | white_mask

    # Find the bounding box for the new flag area
    rows_cropped = np.any(combined_mask_cropped, axis=1)
    cols_cropped = np.any(combined_mask_cropped, axis=0)

    # Get the bounding box coordinates for the cropped image
    top_cropped, bottom_cropped = np.where(rows_cropped)[0][[0, -1]]
    left_cropped, right_cropped = np.where(cols_cropped)[0][[0, -1]]

    # Crop the image again based on the new bounding box
    final_cropped_img = cropped_img_array[top_cropped:bottom_cropped+1, left_cropped:right_cropped+1]

    # Convert the final cropped image back to a Pillow Image
    final_cropped_pil_img = Image.fromarray(final_cropped_img)

    # Save the final cropped image as 'prob5_img2.jpg'
    final_cropped_pil_img.save('prob5_img2.jpg')

    # Split the image into upper and lower halves
    height, width, _ = final_cropped_img.shape
    upper_half = final_cropped_img[:height // 2, :, :]
    lower_half = final_cropped_img[height // 2:, :, :]

    # Calculate the average color intensity (mean pixel value) of the Green and Blue channels for both halves
    upper_half_avg_green_blue = np.mean(upper_half[:, :, 1:], axis=(0, 1))  # Average of Green and Blue for upper half
    lower_half_avg_green_blue = np.mean(lower_half[:, :, 1:], axis=(0, 1))  # Average of Green and Blue for lower half

    # Calculate the average of Green and Blue values for both halves
    upper_half_avg = np.mean(upper_half_avg_green_blue)  # Average of Green and Blue for upper half
    lower_half_avg = np.mean(lower_half_avg_green_blue)  # Average of Green and Blue for lower half

    # Compare the average intensities to determine if it's Indonesia or Poland
    if upper_half_avg < lower_half_avg:  # Check if the upper half has lower average intensity
        print("The flag is Indonesian.")
    else:
        print("The flag is Polish.")

# Test the function with your image path
image_path = "prob5_img1.jpg"  # Replace with the actual path of the image
crop_flag(image_path)


The flag is Polish.
